In [2]:
!pip install -U finance-datareader

In [3]:
import FinanceDataReader as fdr
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [4]:
# 10Y Treasury Yield (FRED)
rf = fdr.DataReader('US10YT', '2025-01-01', '2025-12-25')['Close']
rf_rate = rf.iloc[-1] / 100  # %수치를 숫자로 변환
print(f"10Y Risk-free rate: {rf_rate:.2%}")

10Y Risk-free rate: 4.14%


In [5]:
import itertools
import pandas as pd
import numpy as np

ratings = ['AAA', 'AA', 'A', 'BBB']
maturities = [3, 5, 7, 10]          # years
coupons = [0.03, 0.04, 0.05]        # coupon rates

rating_spread = {
    'AAA': 0.002,
    'AA':  0.004,
    'A':   0.007,
    'BBB': 0.012
}

In [ ]:
universe = []

for r, m, c in itertools.product(ratings, maturities, coupons):
    universe.append({
        'Rating': r,
        'Maturity': m,
        'Coupon': c,
        'Spread': rating_spread[r]
    })

bonds = pd.DataFrame(universe)
print(bonds.head())
print(f"Total bonds in universe: {len(bonds)}")

  Rating  Maturity  Coupon  Spread
0    AAA         3    0.03   0.002
1    AAA         3    0.04   0.002
2    AAA         3    0.05   0.002
3    AAA         5    0.03   0.002
4    AAA         5    0.04   0.002
Total bonds in universe: 48


In [7]:
def bond_price(face, coupon, maturity, discount_rate):
    price = 0
    for t in range(1, maturity + 1):
        price += face * coupon / (1 + discount_rate) ** t
    price += face / (1 + discount_rate) ** maturity
    return price

In [8]:
FACE = 100

prices = []
yields = []
durations = []

for _, b in bonds.iterrows():
    y = rf_rate + b['Spread']
    p = bond_price(FACE, b['Coupon'], b['Maturity'], y)
    prices.append(p)
    yields.append(b['Coupon'])
    durations.append(b['Maturity'])  # 단순 duration proxy

bonds['Price'] = prices
bonds['Yield'] = yields
bonds['Duration'] = durations

print(bonds)

   Rating  Maturity  Coupon  Spread       Price  Yield  Duration
0     AAA         3    0.03   0.002   96.315995   0.03         3
1     AAA         3    0.04   0.002   99.073484   0.04         3
2     AAA         3    0.05   0.002  101.830972   0.05         3
3     AAA         5    0.03   0.002   94.108090   0.03         5
4     AAA         5    0.04   0.002   98.518202   0.04         5
5     AAA         5    0.05   0.002  102.928314   0.05         5
6     AAA         7    0.03   0.002   92.079884   0.03         7
7     AAA         7    0.04   0.002   98.008114   0.04         7
8     AAA         7    0.05   0.002  103.936344   0.05         7
9     AAA        10    0.03   0.002   89.342845   0.03        10
10    AAA        10    0.04   0.002   97.319757   0.04        10
11    AAA        10    0.05   0.002  105.296668   0.05        10
12     AA         3    0.03   0.004   95.780455   0.03         3
13     AA         3    0.04   0.004   98.527554   0.04         3
14     AA         3    0.

In [9]:
y = bonds['Yield'].values
D = bonds['Duration'].values

def portfolio_yield(w):
    return np.dot(w, y)

def neg_portfolio_yield(w):
    return -portfolio_yield(w)

constraints = [
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # 0되는 조건
    {'type': 'ineq', 'fun': lambda w: 7 - np.dot(w, D)}  # 양수되는 조건
]

bounds = [(0, 0.5)] * len(bonds)
w0 = np.ones(len(bonds)) / len(bonds)

res = minimize(
    neg_portfolio_yield,
    w0,
    method='SLSQP',
    bounds=bounds,
    constraints=constraints)

In [11]:
bonds

,Rating,Maturity,Coupon,Spread,Price,Yield,Duration
0,AAA,3,0.03,0.002,96.315995,0.03,3
1,AAA,3,0.04,0.002,99.073484,0.04,3
2,AAA,3,0.05,0.002,101.830972,0.05,3
3,AAA,5,0.03,0.002,94.108090,0.03,5
4,AAA,5,0.04,0.002,98.518202,0.04,5
5,AAA,5,0.05,0.002,102.928314,0.05,5
6,AAA,7,0.03,0.002,92.079884,0.03,7
7,AAA,7,0.04,0.002,98.008114,0.04,7
8,AAA,7,0.05,0.002,103.936344,0.05,7
9,AAA,10,0.03,0.002,89.342845,0.03,10


In [12]:
weights = pd.Series(res.x, index=bonds.index)
print("Optimal Weights")
print(weights.round(3))

print(f"Portfolio Yield: {portfolio_yield(res.x):.2%}")
print(f"Portfolio Duration: {np.dot(res.x, D):.2f}")

Optimal Weights
0     0.000
1     0.000
2     0.063
3     0.000
4     0.000
5     0.062
6     0.000
7     0.000
8     0.062
9     0.000
10    0.000
11    0.062
12    0.000
13    0.000
14    0.062
15    0.000
16    0.000
17    0.062
18    0.000
19    0.000
20    0.063
21    0.000
22    0.000
23    0.062
24    0.000
25    0.000
26    0.062
27    0.000
28    0.000
29    0.062
30    0.000
31    0.000
32    0.063
33    0.000
34    0.000
35    0.062
36    0.000
37    0.000
38    0.063
39    0.000
40    0.000
41    0.062
42    0.000
43    0.000
44    0.062
45    0.000
46    0.000
47    0.062
dtype: float64
Portfolio Yield: 5.00%
Portfolio Duration: 6.25


In [ ]:

import requests

# 호출할 API URL 입력
url = 'https://apis.data.go.kr/1160100/service/GetBondSecuritiesInfoService'

# 요청사항(데이터를 불러오기 위해 반드시 입력해야할 사항) 입력
params ={'serviceKey' : '개인 Service Key', 
         'pageNo' : '1',
         'numOfRows' : '1000',
         'dataType' : 'json',
         'base_date' : '20250111',
         'base_time' : '0600',
         'nx' : '55',
         'ny' : '127' }

# 데이터 호출
response = requests.get(url, params=params)
print(response.text)

API not found

